In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import folium

In [2]:
traffic_Seoul = pd.read_csv("../Data/newSeoul_2005_2019.csv")
traffic_Seoul.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2005,1,종로구,93,2,138
1,2005,2,종로구,84,3,125
2,2005,3,종로구,117,0,142
3,2005,4,종로구,138,2,212
4,2005,5,종로구,145,2,207


In [3]:
# 2019년 데이터만 추출하기
traffic_Seoul_2019 = traffic_Seoul[traffic_Seoul['년도'] == 2019]
traffic_Seoul_2019

,년도,월,자치구명,발생건수,사망자수,부상자수
4200,2019,1,종로구,87,1,125
4201,2019,2,종로구,66,1,84
4202,2019,3,종로구,87,2,122
4203,2019,4,종로구,85,0,131
4204,2019,5,종로구,112,1,158
...,...,...,...,...,...,...
4495,2019,8,강동구,127,2,175
4496,2019,9,강동구,98,2,137
4497,2019,10,강동구,108,1,144
4498,2019,11,강동구,146,0,199


In [4]:
# index 정리하기(0번부터 순서대로 index 번호 정리하기)
traffic_Seoul_2019.reset_index(drop=True, inplace=True)
traffic_Seoul_2019

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2019,1,종로구,87,1,125
1,2019,2,종로구,66,1,84
2,2019,3,종로구,87,2,122
3,2019,4,종로구,85,0,131
4,2019,5,종로구,112,1,158
...,...,...,...,...,...,...
295,2019,8,강동구,127,2,175
296,2019,9,강동구,98,2,137
297,2019,10,강동구,108,1,144
298,2019,11,강동구,146,0,199


In [5]:
# 자치구별 발생건수, 사망자수, 부상자수 집계하기
traffic_anal = pd.pivot_table(traffic_Seoul_2019, index='자치구명', aggfunc=sum)
traffic_anal.head()

,년도,발생건수,부상자수,사망자수,월
자치구명,,,,,
강남구,24228,3722,5182,14,78
강동구,24228,1414,1910,11,78
강북구,24228,1277,1706,7,78
강서구,24228,1829,2491,20,78
관악구,24228,1363,1755,10,78


In [6]:
## 년도와 월은 제거하여 DataSet 만들기
traffic_anal = traffic_anal[['발생건수', '부상자수', '사망자수']]
traffic_anal.head()

,발생건수,부상자수,사망자수
자치구명,,,
강남구,3722,5182,14
강동구,1414,1910,11
강북구,1277,1706,7
강서구,1829,2491,20
관악구,1363,1755,10


In [9]:
seoul_limit = pd.read_csv("../Data/seoul.csv", encoding='euc-kr')
seoul_limit.columns = ['자치구명','lon', 'lat']
seoul_limit.head()

,자치구명,lon,lat
0,강남구,127.0475,37.51731
1,강동구,127.1238,37.53013
2,강북구,127.0255,37.63975
3,관악구,126.9515,37.47834
4,구로구,126.8875,37.49547


In [10]:
data_result = pd.merge(traffic_anal, seoul_limit, on='자치구명')
data_result.head()

,자치구명,발생건수,부상자수,사망자수,lon,lat
0,강남구,3722,5182,14,127.0475,37.51731
1,강동구,1414,1910,11,127.1238,37.53013
2,강북구,1277,1706,7,127.0255,37.63975
3,강서구,1829,2491,20,126.8496,37.55094
4,관악구,1363,1755,10,126.9515,37.47834


In [11]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     int64  
 2   부상자수    25 non-null     int64  
 3   사망자수    25 non-null     int64  
 4   lon     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 1.4+ KB


In [12]:
# folium을 사용하기 위해서 int를 float으로 변환해야 됨
data_result['발생건수'] = data_result['발생건수'].astype(float)
data_result['부상자수'] = data_result['부상자수'].astype(float)
data_result['사망자수'] = data_result['사망자수'].astype(float)

In [13]:
# 변환한 것 확인
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     float64
 2   부상자수    25 non-null     float64
 3   사망자수    25 non-null     float64
 4   lon     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.4+ KB


In [14]:
# 자치구의 구청을 표시하기
data_result

,자치구명,발생건수,부상자수,사망자수,lon,lat
0,강남구,3722.0,5182.0,14.0,127.0475,37.51731
1,강동구,1414.0,1910.0,11.0,127.1238,37.53013
2,강북구,1277.0,1706.0,7.0,127.0255,37.63975
3,강서구,1829.0,2491.0,20.0,126.8496,37.55094
4,관악구,1363.0,1755.0,10.0,126.9515,37.47834
5,광진구,973.0,1316.0,3.0,127.0824,37.53862
6,구로구,1529.0,2096.0,13.0,126.8875,37.49547
7,금천구,920.0,1353.0,9.0,126.8957,37.45708
8,노원구,1703.0,2302.0,8.0,127.0565,37.65437
9,도봉구,852.0,1163.0,4.0,127.0478,37.66873


In [20]:
data_result.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24],
           dtype='int64')

In [19]:
# Map 만들기
seoul_map = folium.Map(
    location=[37.55, 126.98], 
    tiles='openstreetmap',
    zoom_start=12
)


# for name, lat, lng in zip(data_result.자치구명, data_result.lat, data_result.lon):
#     folium.Marker([lat, lng], popup=name).add_to(seoul_map)


# # 대학교 위치정보를 CircleMarker로 표시
# for name, lat, lng in zip(data_result.자치구명, data_result.lat, data_result.lon):
#     folium.CircleMarker([lat, lng],
#                         radius=10,  # 원의 반지름
#                         color='brown',  # 원 둘레 색상
#                         fill=True,   # 컬러 채우기
#                         fill_color='purple',     # 원을 채우는 색
#                         fill_opacity=0.7    # 투명도(숫자가 높아질수록 불투명해짐)
#     ).add_to(seoul_map)


for n in data_result.index:
    folium.Marker(
        [data_result['lat'][n], data_result['lon'][n]],
        popup=data_result['자치구명'][n]
        
    ).add_to(seoul_map)

seoul_map

In [33]:
# Map 만들기
seoul_map = folium.Map(
    location=[37.55, 126.98], 
    tiles='openstreetmap',
    zoom_start=12
)


for n in data_result.index:
    folium.Marker(
        [data_result['lat'][n], data_result['lon'][n]],
        popup=data_result['자치구명'][n]
        
    ).add_to(seoul_map)
    
    
    # 발생건수에 따라 원의 크기를 다르게 한다.(원이 크면 발생건수도 크다)
    folium.CircleMarker(
        [data_result['lat'][n], data_result['lon'][n]],
        radius=data_result['발생건수'][n] / 100,
        color = 'blue',
        fill_color = 'sky blue',
        fill=True,
        fill_opacity=0.3
    ).add_to(seoul_map)
    
    folium.CircleMarker(
        [data_result['lat'][n], data_result['lon'][n]],
        radius=data_result['사망자수'][n],
        color = 'red',
        fill_color = 'red',
        fill=True,
        fill_opacity=0.3
    ).add_to(seoul_map)

seoul_map